<a href="https://colab.research.google.com/github/maorgreshler/NN_google_colab/blob/main/overfitcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import io
from google.colab import files

# Upload the file
uploaded = files.upload()

# Get the actual name of the uploaded file
file_name = list(uploaded.keys())[0]

# Read the file into a DataFrame
df = pd.read_csv(io.BytesIO(uploaded[file_name]))
print(df.head())

Saving overfit_test.csv to overfit_test (8).csv
   x_position  y_position     theta   velocity  yaw_angle  yaw_rate  \
0    6.949054    0.308962 -0.457965  17.818398  -1.001454 -0.513283   
1   -7.417301    9.538423 -1.689320   2.633025  -2.051632  0.188853   
2   -2.801338   -2.354316  2.264059  13.642371   0.328596  0.843307   
3    3.412298   -9.167707  2.944074   9.499971  -0.208473 -0.586376   
4   -2.080115   -8.289799 -2.113399  16.085139  -0.372160  0.091362   

   slip_angle  front_wheel  back_wheel  acceleration  angular_acceleration  \
0    0.114116     0.593264   -0.586801     -4.065305              0.198227   
1   -0.123249    -0.548003   -0.350034     -2.401921             -0.742328   
2    0.282142     0.647917   -0.385001      1.944356              0.092822   
3   -0.504685     0.707749   -0.016458     -4.420800             -0.834798   
4    0.421701     0.466814    0.614045     -3.783537             -0.060659   

   delta_x_position  delta_y_position  delta_theta  delt

In [11]:
input_columns = ["velocity", "yaw_angle", "yaw_rate", "slip_angle", "front_wheel", "back_wheel" , "acceleration", "angular_acceleration"]

output_columns = ["delta_x_position", "delta_y_position", "delta_theta", "delta_velocity", "delta_yaw_angle",
                  "delta_yaw_rate", "delta_slip_angle", "delta_front_wheel", "delta_back_wheel"]

input_data = df[input_columns]
output_data = df[output_columns]

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert pandas DataFrames to numpy arrays
X = input_data.values
y = output_data.values
# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, define loss function and optimizer
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = MLP(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/200], Loss: 0.8203
Epoch [20/200], Loss: 0.5844
Epoch [30/200], Loss: 0.3582
Epoch [40/200], Loss: 0.1930
Epoch [50/200], Loss: 0.0897
Epoch [60/200], Loss: 0.0364
Epoch [70/200], Loss: 0.0111
Epoch [80/200], Loss: 0.0022
Epoch [90/200], Loss: 0.0008
Epoch [100/200], Loss: 0.0003
Epoch [110/200], Loss: 0.0001
Epoch [120/200], Loss: 0.0000
Epoch [130/200], Loss: 0.0000
Epoch [140/200], Loss: 0.0000
Epoch [150/200], Loss: 0.0000
Epoch [160/200], Loss: 0.0000
Epoch [170/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [190/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


In [13]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

#my overfit!!!
# Evaluate the model
model.eval()
with torch.no_grad():
    train_outputs = model(X_train_tensor)
    train_loss = criterion(train_outputs, y_train_tensor)
    print(f'Train Loss: {train_loss.item():.4f}')


Test Loss: 0.7416
Train Loss: 0.0000


In [14]:
# Make predictions
with torch.no_grad():
    predictions = model(X_test_tensor)
    predictions = scaler_y.inverse_transform(predictions.numpy())

print("Predictions:")
print(predictions)

Predictions:
[[-1.4903474e-02 -2.9570043e-02 -6.2479352e-04  4.9958151e-02
  -1.7154354e-03  1.6179634e-02  1.2478869e-03  1.9338973e+00
  -5.6205654e+00]]


In [15]:
#my overfit!!!
# Make predictions
with torch.no_grad():
    predictions = model(X_train_tensor)
    predictions = scaler_y.inverse_transform(predictions.numpy())

print("Predictions:")
import numpy as np
np.set_printoptions(suppress=True, precision=6)
print(predictions[0])

Predictions:
[ 0.01461   0.021399 -0.        0.084646 -0.001934 -0.006616 -0.007681
 -0.635987 -5.106723]
